In [103]:
import matplotlib.pyplot as plt
import numpy as np
from operator import ge, le
import copy
DIRECTIONS = ['up', 'down', 'left', 'right']

        
class Node:
    
    def __init__(self, x, y):
        
        self.x = x
        self.y = y
        self.location = [x, y]
        for dir in DIRECTIONS:
            self.__setattr__(dir, None)
        
    def link(self, other, weight):
        direction = self._compute_direction(other.location)
        self.__setattr__(direction, (other, weight))
        direction = other._compute_direction(self.location)
        other.__setattr__(direction, (other, weight))
        
    def _compute_direction(self, xy):
        x, y = xy
        if self.x < x:
            assert self.y == y
            return 'left'
        if self.x > x:
            assert self.y == y
            return 'right'
        if self.y < y:
            assert self.x == x
            return 'down'
        if self.y > y:
            assert self.x == x
            return 'up'
        raise TypeError('could not link (%s %s) to (%s %s)' % (*self.location, x, y))
    
    def __repr__(self):
        return 'Node(%s, %s)' % (self.x, self.y)

class Particle:
    
    def __init__(self, init_velocity, init_posn, tail_len=25):
        
        self.posn = init_posn
        self.velocity = init_velocity
        self.accelleration = None
        self.node = None
        self.done = False
        self.tail = [self.posn]
        self.tail_len = tail_len
        self.stepsize = 1
    
    def update_tail(self, pt):
        self.tail.append(pt)
        if len(self.tail) > self.tail_len:
            self.tail = self.tail[1:]

    def choose_next_step(self):
        if self.frozen:
            return 
        choices = [self.node.__getattr__(dir) for dir in DIRECTIONS if self.node.__getattr__(dir) is not None]
        if not len(choices):
            self.kill()
            return 
        direction = np.random.choice(choices)
        weight, node = self.node.__getattr__(direction)[0]
        self.node = node
        self.accelleration = -weight
        return direction

    def transition(self, next):
        
        if self.frozen:
            return 
        
        stepsize = self.stepsize
        
        if self.done:
            return
  
        self.posn = self.posn + self.velocity * stepsize
        self.velocity = min(self.velocity - self.accelleration * stepsize, 0)
        self.update_tail(self.posn)
        if self.velocity <= 0:
            self.kill()
        
        self.done = self.check_done()
        
    def kill(self):
        if self.frozen:
            return 
        self.frozen = True

    
class Graph:
    
    def __init__(self, nodes, particles):
        self.nodes = nodes
        self.particles = particles
        self.stepsize = 0
    
    def transition(self, particles=None):
        if particles is None: particles = self.particles
        for particle in particles:
            if particle.done:
                direction = particle.choose_next_step()
                next = particle.__getattr__(direction)
                try: # might get killed during choose step
                    particle.transition(next)
                except:
                    pass
            else:
                particle.transition(next)
    
    def set_global_stepsize(self, stepsize=None):
        if stepsize is None:
            stepsize = self.stepsize
        for particle in self.particles:
            particle.stepsize = stepsize
        self.stepsize = stepsize
                
    def draw(self, i):
        raise NotImplementedError

class RandomGridGraph(Graph):
    
    def __init__(self, size, numparticles, particle_posn=None, dilation=10):
        super(Graph, self).__init__()
        
        self.size = size
        self.dilation = dilation
        self.dim = [size[0] * dilation - 5, size[1] * dilation - 5]
        
        node_array = self._gennodes()
        self.nodes = node_array
        
        particles = []
        for i in range(numparticles):
            particles.append(self._genparticle(particle_posn))
        
        self.particles = particles
        self.init_canvas()
    
    def _gennodes(self):
        out_array = [[Node(x * self.dilation, y * self.dilation) for x in range(self.size[0])]
                     for y in range(self.size[1])]
        self._link_neighbors(out_array)
        return out_array

    def _link_neighbors(self, array):
        def link(array, x, y):
            weight = np.random.uniform(0, 3)
            neighbors = []
            if y + 1 < len(array[0]):
                neighbors.append((x, y + 1))
            if y - 1 >= 0:
                neighbors.append((x, y - 1))
            if x + 1 < len(array):
                neighbors.append((x + 1, y))
            if x - 1 >= 0:
                neighbors.append((x - 1, y))
            for n in neighbors:
                array[x][y].link(array[n[0]][n[1]], weight)
                
        for x in range(len(array)):
            if x % 2 == 0:
                for y in range(0, len(array[0]) - 1, 2):
                    link(array, x, y)
            else:
                for y in range(1, len(array) - 1, 2):
                    link(array, x, y)
                    
    def _genparticle(self, posn):
        if isinstance(posn, (tuple, list)):
            x, y = posn
        else:
            x = np.random.randint(0, self.size[0] - 1)
            y = np.random.randint(0, self.size[1] - 1)
        target_node = self.nodes[x][y]
        velocity = np.random.randint(10, 25)
        particle = Particle(velocity, (x, y))
        particle.node = target_node
        return particle
    
    def init_canvas(self):
        self.canvas = Canvas(self.dim, len(self.particles))
        self.canvas.data = [p.tail for p in self.particles]
    
    def draw(self, i):
        self.canvas.draw()
        self.transition()
        return self.canvas.lines
        
class Canvas:
    def __init__(self, dims, numparticles):
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=dims[0], ylim=dims[1])
        self.lines = [self.ax.plot([],[], lw=2) for i in range(numparticles)]
        self.data = []
    
    def init(self):
        for line in self.lines:
            line[0].set_data([], [])
        return [line[0] for line in self.lines]
    
    def draw(self):
        for i, datum in enumerate(self.data):
            x = [d[0] for d in datum]
            y = [d[1] for d in datum]
            self.lines[i][0].set_data(x, y)
             

In [104]:
import matplotlib
from matplotlib import animation

matplotlib.use('TkAgg')

graph = RandomGridGraph((3, 3), 30)
graph.set_global_stepsize(0.1)

anim = animation.FuncAnimation(graph.canvas.fig, graph.draw, 
                               init_func=graph.canvas.init, frames=300, interval=20, blit=True)

anim.save('anim1.gif', fps=30, writer='imagemagick')



MovieWriter stderr:
convert-im6.q16: unexpected end-of-file `-': No such file or directory @ error/rgb.c/ReadRGBImage/239.



CalledProcessError: Command '['convert', '-size', '432x288', '-depth', '8', '-delay', '3.3333333333333335', '-loop', '0', 'rgba:-', 'anim1.gif']' returned non-zero exit status 1.